# Integrating Python, SQL, and Tableau
Data scientists and analysts have many tools at their disposal. Though there is often overlap between the capabilities of various tools, each one has its own set of strenghts and weaknesses. Rather than having to choose one, it's best to integrate them all together so that we can get the best of both worlds. 

- **Python** is a powerful scripting language with libraries that make it ideal for performing sophisticated machine learning analyses. It also has libraries that allow for keeping limited amounts of information in memory for those analyses, as well as for providing basic visualizations of the data.
- **SQL** is a database tool that is manages vast quantities of data, as well as their relationships. Though not an analytical nor visualization tool, it provides the means to quickly and easily get to the relevant subsets of data required.
- **Tableau** is a visualization tool which are much more suitable for presentation purposes. Though it is possible to construct data from scratch and compute values across rows and columns, it normally just reads data from csv or SQL. 

Through integration, we can get the analytical prowess of Python, the data management sophistication of SQL, and the beatification of Tableau to present the findings of data. We will show how this is accomplished through a simple analysis: predicting absenteeism.

## Project Overview: Predicting Absenteeism
There are many reasons a person might be unable to work. Some are within a company's control, and some aren't. For example, a company might have a high-stress culture that causes people to get sick more often. On the other hand, a person who is pregnant is going to miss work for an extended period irregardless of company behavior. A firm is going to need to respond to these situations in different ways. If we can predict when a person is likely to miss time from work and understand why, it will allow the company to be more proactive in dealing with it. 

We start by using OSEMN framework to complete the analysis, build our machine learning model, and prepare it for deployment.

## Obtaining the Data
We start by loading the data in the file **absenteeism_data.csv**, along with relevant libraries. We use the *pandas_profiling* module to provide a complete overview of our data. It will provide necessary insights to identify null values, duplicate rows, correlation coefficients & heat maps, etc. 

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt

raw_data = pd.read_csv('absenteeism_data.csv')

data = raw_data.copy()
data.profile_report()

### Initial Observations
Target variable will be **Absenteeism_Time_in_Hours** for a given day. 

The **Reason_for_Absence** is actually a categorical column, but it is difficult to say whether or not this will be an effective predictor. We do not generally know the reason until after a person has already called out. We will treat it as a predictor for the time being, but depending on the strength of the relationship, it is likely we will need to develop a new model to predict if a given reason is likely to be exhibited by a person. For example, if a reason is sickness, we would want to know if there is a way to predict that someone might get sick. Though it may make the model stronger, there are practical implications to consider. 

## Scrubbing the Data
### Warnings
By looking at the *Warnings* section, we can get a sense of what will need to be cleaned up.

- Remove duplicate rows
- Rows with zeros in **Absenteeism_Time_in_Hours** column will be dropped
- Get dummies for **Reason_for_Absence** column and group in the following manner

 - **Reasons 1-14**: Sickness
 - **Reasons 15-17**: Pregnancy
 - **Reasons 18-21**: Injury
 - **Reasons 22-28**: Doctor Appointment

In addition, the **ID** column is not necessary for the analysis, so this column will be dropped.

In [2]:
# Drop duplicate rows
data.drop_duplicates(inplace=True)

# Drop rows with 0 hours absent
df = data[data['Absenteeism_Time_in_Hours'] != 0]

# Reset index
df = df.reset_index(drop=True)

# Create reason groups
reasons = {'Sickness': list(range(1, 15)), 
           'Pregnancy': list(range(15, 18)), 
           'Injury': list(range(18, 22)), 
           'Doctors_Appt': list(range(22, 29))}

# Get dummies
for reason in reasons.keys():
    df[reason] = [1 if num in reasons[reason] else 0 for num in df['Reason_for_Absence']]
    
# Drop Reason & ID columns
df.drop(['Reason_for_Absence', 'ID'], axis=1, inplace=True)
df.head()

,Date,Transportation_Expense,Distance_to_Work,Age,Daily_Work_Load_Average,Body_Mass_Index,Education,Children,Pets,Absenteeism_Time_in_Hours,Sickness,Pregnancy,Injury,Doctors_Appt
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
2,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
3,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1
4,10/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1


### Time of Week / Year
Does day of the week matter? Are individuals more likely to call out on Monday or Friday? What about time of year? We will extract the day of the week and month name from the date column to see if that makes a difference.

In [3]:
# Convert date column to datetime format
df.Date = pd.to_datetime(df.Date)

# Get day name dummies
df['Weekday'] = [date.weekday() for date in df.Date]
df['Month'] = [month.month for month in df['Date']]

df.drop('Date', axis=1, inplace=True)
df.head()

,Transportation_Expense,Distance_to_Work,Age,Daily_Work_Load_Average,Body_Mass_Index,Education,Children,Pets,Absenteeism_Time_in_Hours,Sickness,Pregnancy,Injury,Doctors_Appt,Weekday,Month
0,289,36,33,239.554,30,1,2,1,4,0,0,0,1,1,7
1,179,51,38,239.554,31,1,0,0,2,0,0,0,1,2,7
2,279,5,39,239.554,24,1,2,0,4,1,0,0,0,3,7
3,289,36,33,239.554,30,1,2,1,2,0,0,0,1,3,7
4,179,51,38,239.554,31,1,0,0,2,0,0,0,1,2,10


### Education
This variable is categorical, representing various levels of education. One (1) indicates high school has been completed, while the remaining indicate that a higher level of education has been completed. The data is highly skewed with a vast majority of employees only finishing high school. It would make sense to make this column binary, with one (1) indicating that the person has education beyond high school, and 0 representing high school education only.

In [4]:
df.Education = [1 if lvl > 1 else 0 for lvl in df.Education]
df.Education.value_counts()

0    539
1    114
Name: Education, dtype: int64

### Target Variable
The firm is primarily concerned about individuals who miss a half day of work or more; missing a couple of hours here or there is not a big deal. So when the the number of hours absent is 4 or more, we will assign a value of 1 to indicate a significant period of absence, otherwise we assign a value 0. 

The column indicating number of hours absent will be kept in case there is a later desire to predict number of hours absent. Selection of the appropriate target variable will take place at the modeling stage.

In [5]:
# Create binary target
absences=[1 if hrs >=4 else 0 for hrs in df['Absenteeism_Time_in_Hours']]
df['Significant_Absence'] = absences
df.Significant_Absence.value_counts()

0    334
1    319
Name: Significant_Absence, dtype: int64

### Write Preprocessed File to CSV
Before writing to csv, put target variables at end of dataframe.

In [6]:
# Move target variables to end
cols = list(df.columns)
cols.pop(cols.index('Absenteeism_Time_in_Hours'))
cols.pop(cols.index('Significant_Absence'))
df = df[cols+['Absenteeism_Time_in_Hours','Significant_Absence']]

df.to_csv('df_preprocessed.csv')

In [7]:
df.head()

,Transportation_Expense,Distance_to_Work,Age,Daily_Work_Load_Average,Body_Mass_Index,Education,Children,Pets,Sickness,Pregnancy,Injury,Doctors_Appt,Weekday,Month,Absenteeism_Time_in_Hours,Significant_Absence
0,289,36,33,239.554,30,0,2,1,0,0,0,1,1,7,4,1
1,179,51,38,239.554,31,0,0,0,0,0,0,1,2,7,2,0
2,279,5,39,239.554,24,0,2,0,1,0,0,0,3,7,4,1
3,289,36,33,239.554,30,0,2,1,0,0,0,1,3,7,2,0
4,179,51,38,239.554,31,0,0,0,0,0,0,1,2,10,2,0
